# Setting up the Environmnet

In [1]:
import boto3, time, json
sess    = boto3.Session()
sm      = sess.client('sagemaker')
region  = sess.region_name
account = boto3.client('sts').get_caller_identity().get('Account')

In [2]:
import sagemaker
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session(boto_session=sess)

In [3]:
model_file_name = "nli_r1"
s3_folder = "nli"

In [4]:
bucket_name = sagemaker_session.default_bucket()
print(bucket_name)

sagemaker-us-west-1-096166425824


In [5]:
prefix = 'torchserve'
registry_name = 'torchserve'

In [1]:
## Using the precreated image created in the Create_sagemaker_torchserve_AMI 

In [11]:
image_label = 'v1'
image = f'{account}.dkr.ecr.{region}.amazonaws.com/{registry_name}:{image_label}'

Sending build context to Docker daemon   7.64GB
Step 1/22 : FROM ubuntu:18.04
 ---> 8e4ce0a6ce69
Step 2/22 : ENV PYTHONUNBUFFERED TRUE
 ---> Using cache
 ---> 6b822023851a
Step 3/22 : RUN apt-get update &&     DEBIAN_FRONTEND=noninteractive apt-get install --no-install-recommends -y     fakeroot     ca-certificates     dpkg-dev     g++     python3-dev     openjdk-11-jdk     curl     vim     && rm -rf /var/lib/apt/lists/*     && cd /tmp     && curl -O https://bootstrap.pypa.io/get-pip.py     && python3 get-pip.py
 ---> Using cache
 ---> 443bc4cfe0b0
Step 4/22 : RUN update-alternatives --install /usr/bin/python python /usr/bin/python3 1
 ---> Using cache
 ---> ceb242ba9abd
Step 5/22 : RUN update-alternatives --install /usr/local/bin/pip pip /usr/local/bin/pip3 1
 ---> Using cache
 ---> 546f8ef08943
Step 6/22 : RUN pip install aiohttp
 ---> Using cache
 ---> 8e12c6f57262
Step 7/22 : RUN pip install allennlp==0.8.4
 ---> Using cache
 ---> 4105bd754583
Step 8/22 : RUN pip install zmq
 ---> 

83644a9b: Pushing  708.3MB/874.1MBPushing  211.2MB/874.1MB

87635259: Pushing  2.268GB/3.184GB

87635259: Pushed   3.206GB/3.184GBPushing   2.59GB/3.184GBPushing  2.908GB/3.184GBPushing  3.005GB/3.184GBv1: digest: sha256:443a1b14982355b41a0965535059f51da823585464f268ddcf5ac333c3eb9b4e size: 4513


# Model Creation

In [13]:
from sagemaker.model import Model
from sagemaker.predictor import RealTimePredictor

model_data = f's3://{bucket_name}/{prefix}/models/{s3_folder}/{model_file_name}.tar.gz'
sm_model_name = 'nli-r1'

torchserve_model = Model(model_data = model_data, 
                         image = image,
                         role  = role,
                         predictor_cls=RealTimePredictor,
                         name  = sm_model_name)

# Model Deployment

In [14]:
endpoint_name = sm_model_name
#Change the initial_instance_count to the number of instances you want to be launched.
predictor = torchserve_model.deploy(instance_type='ml.c5.2xlarge',
                                    initial_instance_count=1,
                                    endpoint_name = endpoint_name)

---------------!